In [236]:
from pathlib import Path


from matplotlib import rcParams
from matplotlib.cm import rainbow
import math
import numpy as np
import pandas as pd


from math import sin, cos, sqrt, atan2, radians


from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn import  linear_model

from collections import Counter

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

#Training
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


pd.set_option("display.max_columns", 100)




/kaggle/input/ml-dataset/ML data/test.csv
/kaggle/input/ml-dataset/ML data/sample_submission.csv
/kaggle/input/ml-dataset/ML data/train.csv
/kaggle/input/fare-classification/meta_data.csv


In [237]:
R = 6373.0

#lable encording
def encoding_label(label):
  if(label=='correct'):
    return 1
  else:
    return 0
#calculate hours
def calculate_hours(time):
    hours=time.total_seconds()
    return hours    
#calculate distance
def calculate_distance(coordinates):

#     print(coordinates['pick_lat'])
#     print(type(coordinates['pick_lat']))
    lat1 = math.radians(coordinates['pick_lat'])
    lon1 = math.radians(coordinates['pick_lon'])
    lat2 = math.radians(coordinates['drop_lat'])
    lon2 = math.radians(coordinates['drop_lon'])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

#Finging outliers
def calculate_outliers(data,features):
    out_liers=[]
    for feature in features:
        column_data= data[[feature]]
#         column_data.head()
        print(feature)
        column_data.sort_values(by=[feature], inplace=True)
        
        q1, q3= np.percentile(column_data,[5,99.9])
        iqr = q3 - q1
#         lower_bound = q1 -(1.5 * iqr) 
        upper_bound = q3 +(1.5 * iqr) 
        print(q1)
        print(q3)
#         outlier_list_col = column_data[(column_data < lower_bound) | (column_data > upper_bound)].index
        outlier_list_col = column_data[ (column_data > upper_bound)].index
        out_liers.extend(outlier_list_col)
    out_liers=Counter(out_liers)
    print(out_liers)
    out_liers = list(index for index,count in out_liers.items() if count > 2)
    column_data.head()
    return out_liers
        




Definition

In [238]:

train_data = pd.read_csv("/kaggle/input/ml-dataset/ML data/train.csv", index_col="tripid")
train_data["label"]=train_data["label"].apply(encoding_label).values
test_data = pd.read_csv("/kaggle/input/ml-dataset/ML data/test.csv", index_col="tripid")
submissionFormatData = pd.read_csv("/kaggle/input/ml-dataset/ML data/sample_submission.csv", index_col="tripid")
#training_data=train_data["label"]
# index_data=test_data["tripid"]
# train_data.head()
# Trainng_feature=train_data[]
# trainng_feature= train_data.loc[:,[0:12]]


# trainng_label.head()
# test_data.head()
# train_data.columns.values
# train_data.dropna()
# train_data.dropna()

Removing outliers

In [239]:
print(len(train_data.index))

17176


In [240]:
train_data=train_data[(train_data['additional_fare']< 5000)&(train_data['duration']< 339351)&(train_data['meter_waiting'] <339312)&(train_data['meter_waiting_fare']<19570)&(train_data['meter_waiting_till_pickup']<8800)&(train_data['fare']< 25097)]

In [241]:
print(len(train_data.index))

16959


In [242]:
#Reemove NAN tuples
train_data.dropna(inplace=True)
#dropNAN columns from test list
NANcolumns=test_data.columns[test_data.isna().any()].tolist()
#remove tuples from submissionData
submissionFormatData.drop(NANcolumns)
test_data.dropna(inplace=True)

In [243]:
# # out_liers= calculate_outliers(train_data, ["duration","meter_waiting","meter_waiting_fare","fare","pick_lat","pick_lon","drop_lat","drop_lon"])
# # out_liers= calculate_outliers(train_data, ["additional_fare","duration","meter_waiting","meter_waiting_fare","fare"])
# out_liers= calculate_outliers(train_data, ["meter_waiting_fare"])
# out_liers
# train_data.drop(out_liers,inplace=True)

Data Preprocessing 

In [244]:
#feature engineering



trainng_label= train_data[["label"]]
trainng_feature= train_data.loc[:,"additional_fare":"fare"]


trainng_feature['pickup_time'] = pd.to_datetime(trainng_feature['pickup_time'],format='%m/%d/%Y %H:%M')
trainng_feature['drop_time'] = pd.to_datetime(trainng_feature['drop_time'],format='%m/%d/%Y %H:%M')
test_data['pickup_time'] = pd.to_datetime(test_data['pickup_time'],format='%m/%d/%Y %H:%M')
test_data['drop_time'] = pd.to_datetime(test_data['drop_time'],format='%m/%d/%Y %H:%M')


trainng_feature['time_difference']=trainng_feature['drop_time']-trainng_feature['pickup_time']
trainng_feature['time_difference']=trainng_feature['time_difference'].apply(calculate_hours).values
test_data['time_difference']=test_data['drop_time']-test_data['pickup_time']
test_data['time_difference']=test_data['time_difference'].apply(calculate_hours).values

#calculate distance

# trainng_feature['time_difference']=trainng_feature['time_difference'].days
# trainng_feature['pyear']=trainng_feature['pickup_time'].dt.year 
# trainng_feature['pmonth']=trainng_feature['pickup_time'].dt.month 
# trainng_feature['pday']=trainng_feature['pickup_time'].dt.day
# trainng_feature['pdayofweek_num']=trainng_feature['pickup_time'].dt.dayofweek 
# trainng_feature['pHour'] = trainng_feature['pickup_time'].dt.hour 
# trainng_feature['pminute'] = trainng_feature['pickup_time'].dt.minute 




trainng_feature=trainng_feature.drop(['pickup_time','drop_time'],axis=1)
test_data=test_data.drop(['pickup_time','drop_time'],axis=1)
# trainng_feature['dyear']=trainng_feature['drop_time'].dt.year 
# trainng_feature['dmonth']=trainng_feature['drop_time'].dt.month 
# trainng_feature['dday']=trainng_feature['drop_time'].dt.day
# trainng_feature['ddayofweek_num']=trainng_feature['drop_time'].dt.dayofweek 
# trainng_feature['dHour'] = trainng_feature['drop_time'].dt.hour 
# trainng_feature['dminute'] = trainng_feature['drop_time'].dt.minute 
# test_data.isnull().sum()
#merginh two dataframes
trainng_feature["distance"]=trainng_feature[['pick_lat','pick_lon','drop_lat','drop_lon']].apply(calculate_distance,axis = 1)
test_data["distance"]=test_data[['pick_lat','pick_lon','drop_lat','drop_lon']].apply(calculate_distance,axis = 1)
# drop columns
trainng_feature=trainng_feature.drop(['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
test_data=test_data.drop(['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
trainng_feature.head()
# trainng_feature.isnull().sum()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,time_difference,distance
tripid,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,270.32,840.0,5.094369
189125358,10.5,791.0,47.0,0.0000,134.0,197.85,780.0,3.169052
189125719,10.5,1087.0,80.0,0.0000,61.0,301.64,1080.0,6.307375
189127273,10.5,598.0,271.0,15.6638,68.0,82.30,600.0,0.862217
189129552,10.5,3407.0,182.0,0.0000,112.0,1065.02,3420.0,24.214638


Check for outliers

In [245]:
# numerical_cols = trainng_feature.columns[trainng_feature.dtypes != "object"].values

In [246]:
# numerical_cols

In [247]:
# numeric_preprocessing_steps = Pipeline([
#     ('standard_scaler', StandardScaler()),
#     ('simple_imputer', SimpleImputer(strategy='mean'))
# ])

# preprocessor = ColumnTransformer(
#     transformers = [
#         ("numeric", numeric_preprocessing_steps, numerical_cols)
#     ],
#     remainder = "drop"
# )

Parameter tuning

In [248]:
rf = RandomForestRegressor(random_state = 42)
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [249]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [250]:
# print(type(trainng_label))
# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestClassifier()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1,scoring ='f1')
# # Fit the random search model
# rf_random.fit( trainng_feature, trainng_label)

In [251]:
rf_random.best_params_

{'n_estimators': 1400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [252]:
estimator=RandomForestClassifier(     
         n_estimators = 1400,
         min_samples_split= 5,
         min_samples_leaf= 1,
         max_features = 'sqrt',
         max_depth = 30,
         bootstrap = True
                                    
                                    
    )

In [253]:
full_pipeline = Pipeline([
#     ("preprocessor", preprocessor),
    ("estimators", estimators),
])

In [254]:
# full_pipeline

Creating the dataset

In [255]:
X_train, X_eval, y_train, y_eval = train_test_split(
    trainng_feature,
    trainng_label,
    test_size=0.3,
    shuffle=True,
    stratify= trainng_label,
    random_state=6
)
X_train.head()

# y_train.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,time_difference,distance
tripid,,,,,,,,
207712310,60.5,1906.0,651.0,37.9533,198.0,353.19,1920.0,0.857686
208656446,10.5,1087.0,23.0,1.3409,201.0,258.61,1080.0,4.258433
201195031,21.0,2203.0,349.0,0.0000,34.0,417.70,2220.0,10.824247
209848270,10.5,601.0,492.0,28.6836,67.0,89.18,600.0,0.035718
194557549,10.5,658.0,50.0,0.0000,47.0,152.58,660.0,2.464937


Train the model

In [256]:
full_pipeline.fit(X_train, y_train)

# Predict on evaluation set
# This competition wants probabilities, not labels
preds = full_pipeline.predict(X_eval)
preds

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [257]:
print("test_probas[0].shape", preds.shape)

test_probas[0].shape (5088, 1)


In [258]:
y_preds = pd.DataFrame(
    {
        "prediction": preds[:, 0]
    },
    index = y_eval.index
)
print("y_preds.shape:", y_preds.shape)
y_preds.head()

y_preds.shape: (5088, 1)


,prediction
tripid,
198065060,1
194256607,1
204527672,1
205281640,1
200225350,1


Test model

In [259]:
f1_score(y_eval, y_preds, average=None)

array([0.672     , 0.97390197])

In [260]:
full_pipeline.fit(trainng_feature, trainng_label)

Pipeline(memory=None,
         steps=[('estimators',
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                        max_depth=67,
                                                                        max_features='auto',
                                                                        max_leaf_nodes=None,
                                                                        max_samples=None,
                                                                        min_impurity_decrease=0.0,
                                                                        min_impurity_split=None,
                                 

In [261]:
test_probas = full_pipeline.predict(test_data)
test_probas

array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [262]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_data.index.values, 
                              submissionFormatData.index.values)


submissionFormatData["prediction"]=test_probas[:, 0]
submissionFormatData.head()

submissionFormatData.to_csv('my_submission.csv', index=True)

Show output

In [263]:
!head my_submission.csv

tripid,prediction
213284604,1
213286352,0
213293973,1
213294622,1
213298687,1
213299545,1
213302332,1
213302671,1
213305594,1
